# Comparison of Search Method
This notebook runs the search techniques based on the attribute sets proposed in this paper

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from sklearn.preprocessing import Imputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from scipy.stats import sem
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import os
import numpy as np
import json

In [2]:
train_sizes = [1,10,100,1000,10000,75000]
test_size = 20000
n_repeats = 25
pick_size = 50

## Load in the Known Data

In [3]:
def load_data(path):
    """Read in a JSON file from disk, and return a DataFrame with columns as attributes and class"""
    
    temp = json.load(open(path))
    data = pd.DataFrame([x['attributes']+[x['class']['measured'],] for x in temp['entries']],
                        columns=temp['attribute-names']+['class',])
    data = data[~data['class'].isnull()]
    return data
data = load_data(os.path.join('..', 'datasets', 'quat-heuslers.json'))

In [4]:
count = len(data)
n_stable = (data['class'] <= 0).sum()
print('Number of stable QHs: %d/%d'%(n_stable, count))

Number of stable QHs: 352/96188


In [5]:
print('Expected number of stable / test split:', n_stable / count * test_size)

Expected number of stable / test split: 73.190002911


Load in OQMD/TH data

In [6]:
other_data = pd.concat([load_data(os.path.join('..', 'datasets', '%s.json'%x)) 
                        for x in ['oqmd-no-heusler', 'heuslers']])

In [7]:
other_data = other_data[np.logical_not(np.any(other_data.isnull(), axis=1))]

## Implementing the Search Strategies
Each of these methods must take two arguements: "known" data for training, and the search space. Both arguments are Pandas data frames, and the method must return `pick_size` number of rows from that dataframe.

In [8]:
tests = dict()

### Random Search
Just pick rows randomly

In [9]:
def random_search(train_data, search_data):
    return search_data.sample(pick_size)

In [10]:
tests['Random\nSearch'] = random_search

### Valence Electron Counting
Prioritize compounds with 18 or 24 electrons

In [11]:
def valence_count(train_data, search_data):
    search_data = search_data.copy()
    search_data['matches'] = np.logical_or(np.isclose(search_data['mean_NValance'], 18.0 / 4),
                                           np.isclose(search_data['mean_NValance'], 24.0 / 4))
    return search_data.sample(len(search_data)).sort_values('matches', ascending=False).iloc[:pick_size]

In [12]:
tests['Electron\nCounting'] = valence_count 

### Machine Learning Method
Train an ML model using the method demonstrated in this work

In [13]:
X_cols = data.columns[:-1]
y_col = data.columns[-1]

In [14]:
model = Pipeline([
    ('imputer', Imputer()),
    ('rf', RandomForestRegressor(n_estimators=100, n_jobs=-1))
])

In [15]:
def ml_method(train_data, test_data):
    # Train the model
    model.fit(train_data[X_cols].astype(np.float32), train_data[y_col])
    
    # Predict stability
    test_data = test_data.copy()
    test_data['prediction'] = model.predict(test_data[X_cols])
    return test_data.sort_values('prediction', ascending=True).iloc[:pick_size]

In [16]:
tests['Ward,\nonly QH'] = ml_method

In [17]:
def ml_with_oqmd(train_data, test_data):
    return ml_method(pd.concat([train_data, other_data]), test_data)

In [18]:
tests['Ward,\nall OQMD'] = ml_with_oqmd

## Running the Test
For those methods we can test here

In [19]:
results = dict([(k,dict()) for k in tests.keys()])
for k,f in tqdm(tests.items(), desc='Method'):
    for train_size in tqdm(train_sizes, desc='Size', leave=False):
        results[k][str(train_size)] = []
        for i in tqdm(range(n_repeats), desc='Test', leave=False):
            train_data, test_data = train_test_split(data, train_size=train_size, test_size=test_size, random_state=i)
            results[k][str(train_size)].append(int(sum(f(train_data, test_data)['class'] <= 0)))
            json.dump(results, open('test-results.json', 'w'))

HBox(children=(IntProgress(value=0, description='Method', max=4), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Size', max=6), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Test', max=25), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Test', max=25), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Test', max=25), HTML(value='')))

...

HBox(children=(IntProgress(value=0, description='Test', max=25), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Test', max=25), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Test', max=25), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Test', max=25), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Test', max=25), HTML(value='')))

## Save all results to disk
These will be analyzed all plotted in another notebook

In [20]:
json.dump(results, open('test-results.json', 'w'))